# 手写并验证滑动相乘实现PyTorch二维卷积

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_23_手写并验证滑动相乘实现PyTorch二维卷积：
    
https://www.bilibili.com/video/BV1dP4y137er/?spm_id_from=pageDriver&vd_source=18e91d849da09d846f771c89a366ed40

torch.nn.Conv2d 官方文档：https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

input = torch.randn(5, 5)  # 卷积输入特征图
kernel = torch.randn(3, 3)  # 卷积核
bias = torch.randn(1)  # 卷积偏置，默认输出通道数目等于1

## step1 用原始的矩阵运算来实现二维卷积(先不考虑batchsize维度和channel维度）

In [11]:
def matrix_multiplication_for_conv2d(input, kernel, bias=0, stride=1, padding=0):
    if padding > 0:
        input = F.pad(input, (padding, padding, padding, padding))
    input_h, input_w = input.shape
    kernel_h, kernel_w = kernel.shape
    
    output_h = (math.floor((input_h - kernel_h)/stride) + 1)  # 卷积输出的高度
    output_w = (math.floor((input_w - kernel_w)/stride) + 1)  # 卷积输出的宽度
    output = torch.zeros(output_h, output_w)  # 初始化输出矩阵
    for i in range(0, input_h-kernel_h+1, stride):  # 对高度维进行遍历
        for j in range(0, input_w-kernel_w+1, stride):  # 对宽度维进行遍历
            region = input[i:i+kernel_h, j:j+kernel_w]  # 取出被核滑动到的区域
            output[int(i/stride), int(j/stride)] = torch.sum(region * kernel) + bias  # 点乘，并赋值给输出位置的元素
            
    return output
# 矩阵运算实现卷积的结果
mat_mul_conv_output = matrix_multiplication_for_conv2d(input, kernel, bias=bias, padding=1, stride=2)
print(mat_mul_conv_output)

tensor([[-0.2715, -0.1478, -1.0462],
        [-0.8441, -3.0159, -1.0775],
        [-1.1408,  0.4017, -1.1233]])


In [12]:
# 调用PyTorch API卷积的结果
pytorch_api_conv_output = F.conv2d(input.reshape((1,1,input.shape[0],input.shape[1])), \
                              kernel.reshape((1,1,kernel.shape[0],kernel.shape[1])),\
                                  padding=1,\
                                  bias=bias, stride=2)
print(pytorch_api_conv_output.squeeze(0).squeeze(0))

tensor([[-0.2715, -0.1478, -1.0462],
        [-0.8441, -3.0159, -1.0775],
        [-1.1408,  0.4017, -1.1233]])


## 验证成功，矩阵乘法实现的卷积跟PyTorch API的结果一致